In [1]:
import os
import sys
import time
import glob
import numpy as np
import logging
import argparse
import tensorflow as tf
import numpy as np
import utils
from tqdm import tqdm
import shutil
from model_search import Network
from architect_graph import Architect

import time
from genotypes import PRIMITIVES
from genotypes import Genotype
from scipy.special import softmax

In [2]:
# tf.enable_eager_execution()

In [3]:
import utils
from genotypes import Genotype

In [4]:
args = {
    "momentum": 0.9,
    "weight_decay": 3e-4,
    "arch_learning_rate": 3e-1,
    "momentum": 0.9,
    "grad_clip": 5,
    "learning_rate": 0.025,
    "learning_rate_decay": 0.97,
    "learning_rate_min": 0.0001,
    "num_batches_per_epoch": 2000,
    
    "unrolled": True,
    "epochs": 10,
    "train_batch_size": 8,
    "eval_batch_size": 8,
    "save": "EXP",
    "init_channels": 3,
    "num_layers": 3,
    "num_classes": 6,
    "crop_size": [8, 8],
    "save_checkpoints_steps": 100,
    "model_dir": 'gs://unet-darts/train-search-ckptss',
    "max_steps": 10000,
    # NEW
    "steps_per_eval": 2,
    "num_train_examples": 16,
    #
    
    "use_tpu": True,
    "use_host_call": True,
    "tpu": 'unet-darts',
    "zone": 'us-central1-f',
    "project": "isro-nas"
}

class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

args = Struct(**args)

### input_fn

In [5]:
def make_inp_fn(filename, mode, batch_size):
    
    def _input_fn(params):
        image_dataset = tf.data.TFRecordDataset(filename)
        W, H = args.crop_size[0], args.crop_size[1]

        # Create a dictionary describing the features.  
        image_feature_description = {
            'train_name': tf.FixedLenFeature([], tf.string),  
            'train_x': tf.FixedLenFeature([], tf.string),
            'train_y': tf.FixedLenFeature([], tf.string),
            'valid_name': tf.FixedLenFeature([], tf.string),  
            'valid_x': tf.FixedLenFeature([], tf.string),
            'valid_y': tf.FixedLenFeature([], tf.string)
        }
        def _parse_image_function(example_proto):
            # Parse the input tf.Example proto using the dictionary above.
            feature= tf.parse_single_example(example_proto, image_feature_description)
            train_x = feature['train_x']
            train_y = feature['train_y']
            valid_x = feature['valid_x']
            valid_y = feature['valid_y']
            train_name = feature['train_name']
            valid_name = feature['valid_name']

            train_x = tf.image.decode_png(train_x, channels=3)
            train_y = tf.image.decode_png(train_y, channels=3)
            valid_x = tf.image.decode_png(valid_x, channels=3)
            valid_y = tf.image.decode_png(valid_y, channels=3)
            
            
            train_x = tf.cast(train_x, tf.float32)
            train_x = tf.image.resize(train_x, (W, H))
            train_y = tf.cast(train_y, tf.float32)
            train_y = tf.image.resize(train_y, (W, H))
            valid_x = tf.cast(valid_x, tf.float32)
            valid_x = tf.image.resize(valid_x, (W, H))
            valid_y = tf.cast(valid_y, tf.float32)
            valid_y = tf.image.resize(valid_y, (W, H))
            
            train_y = train_y[:, :, 0]
            train_y = tf.expand_dims(train_y, axis=-1)
            
            valid_y = valid_y[:, :, 0]
            valid_y = tf.expand_dims(valid_y, axis=-1)
            
            return ((train_x, valid_x), (train_y, valid_y))

        dataset = image_dataset.map(_parse_image_function)
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # indefinitely
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)
        else:
            num_epochs = 1 # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size,  drop_remainder=True)

        return dataset
    
    return _input_fn

In [6]:
# %%bash
# mkdir ../../datasets/train_search
# rm ../../datasets/train_search/*.tfrecords
# gsutil cp -r gs://unet-darts/datasets/train-search/*.tfrecords ../../datasets/train_search

In [7]:
def make_inp_fn2(filename, mode, batch_size):
    
    def _input_fn(params):
        W, H = args.crop_size[0], args.crop_size[1]
        NUM_IMAGES = 20
        x_train = np.random.randint(0, 256, (NUM_IMAGES, W, H, 3)).astype(np.float32)
        y_train = np.random.randint(0, args.num_classes, (NUM_IMAGES, W, H, 1)).astype(np.float32)
        x_valid = np.random.randint(0, 256, (NUM_IMAGES, W, H, 3)).astype(np.float32)
        y_valid = np.random.randint(0, args.num_classes, (NUM_IMAGES, W, H, 1)).astype(np.float32)

        ds = (x_train, x_valid), (y_train, y_valid)
        dataset = tf.data.Dataset.from_tensor_slices(ds)
        num_epochs = None # indefinitely
        dataset = dataset.shuffle(buffer_size = 10 * batch_size)

        dataset = dataset.repeat(num_epochs).batch(batch_size, drop_remainder=True)
        return dataset
    
    return _input_fn

### Model function

In [8]:
class GeneSaver(tf.estimator.SessionRunHook):
    def __init__(self, model, alphas_normal, alphas_reduce):
        self.model = model
        self.alphas_normal = alphas_normal
        self.alphas_reduce = alphas_reduce
    
    def begin(self):
        self.global_step = tf.train.get_or_create_global_step()
        
    def end(self, session):
        alphas_normal, alphas_reduce = session.run([self.alphas_normal, self.alphas_reduce])
        alphas_normal = softmax(alphas_normal)
        alphas_reduce = softmax(alphas_reduce)
        
        genotype = self.model.genotype(alphas_normal, alphas_reduce)        
        self.global_step = session.run(self.global_step)
        
        filename = 'final_genotype.{}'.format((self.global_step))
        tf.logging.info("Saving Genotype for step: {}".format(str(self.global_step)))
        utils.write_genotype(genotype, filename)

In [15]:
def model_fn(features, labels, mode, params):
    criterion = tf.losses.softmax_cross_entropy
    model = Network(C=args.init_channels, net_layers=args.num_layers, criterion=criterion, num_classes=args.num_classes)
    global_step = tf.train.get_global_step()
    learning_rate_min = tf.constant(args.learning_rate_min)
    
    learning_rate = tf.train.exponential_decay(
        args.learning_rate,
        global_step,
        decay_rate=args.learning_rate_decay,
        decay_steps=args.num_batches_per_epoch,
        staircase=True,
    )
    
    lr = tf.maximum(learning_rate, learning_rate_min)
    
    optimizer = tf.train.MomentumOptimizer(lr, args.momentum)
    criterion = tf.losses.softmax_cross_entropy
    
    if(args.use_tpu):
        optimizer = tf.tpu.CrossShardOptimizer(optimizer)
        
    eval_hooks = None
    loss = None
    train_op = None
    eval_metric_ops = None
    prediction_dict = None
    export_outputs = None
    host_call = None
    # 2. Loss function, training/eval ops
    if mode == tf.estimator.ModeKeys.TRAIN:
        (x_train, x_valid) = features
        (y_train, y_valid) = labels

        preds = model(x_train)
        architect = Architect(model, args)
        # architect step
        architect_step = architect.step(input_train=x_train,
                                        target_train=y_train,
                                        input_valid=x_valid,
                                        target_valid=y_valid,
                                        unrolled=args.unrolled,
                                        )


        w_var = model.get_thetas()
        loss = model._loss(preds, y_train)
        grads = tf.gradients(loss, w_var)
        clipped_gradients, norm = tf.clip_by_global_norm(grads, args.grad_clip)
        train_op = optimizer.apply_gradients(zip(clipped_gradients, w_var), global_step=tf.train.get_global_step())
        
        b, w, h, c = y_train.shape
        y = tf.reshape(tf.cast(y_train, tf.int64), (b, w, h))
        y = tf.one_hot(y, args.num_classes, on_value=1.0, off_value=0.0)
        
        if args.use_host_call:
            def host_call_fn(global_step, learning_rate, loss, y, preds):
                # Outfeed supports int32 but global_step is expected to be int64.
                print(loss)
                global_step = tf.reduce_mean(global_step)
                global_step = tf.cast(global_step, tf.int64)
                miou, conf_mat = tf.metrics.mean_iou(
                    labels=y,
                    predictions=tf.round(preds),
                    num_classes=args.num_classes)
                
                acc = tf.metrics.accuracy(labels=y, 
                                              predictions=tf.round(preds))
                with (tf.contrib.summary.create_file_writer(args.model_dir).as_default()):
                    with tf.contrib.summary.always_record_summaries():
                        tf.contrib.summary.scalar(
                            'learning_rate', tf.reduce_mean(learning_rate),
                            step=global_step)
                        tf.contrib.summary.scalar(
                            'loss', tf.reduce_mean(loss),step=global_step)
                        tf.contrib.summary.scalar(
                            'acc', acc,step=global_step)
                        tf.contrib.summary.scalar(
                            'miou', miou, step=global_step)
                return tf.contrib.summary.all_summary_ops()

            global_step_t = tf.reshape(global_step, [1])
            learning_rate_t = tf.reshape(learning_rate, [1])
            loss_t = tf.reshape(loss, [1])
            host_call = (host_call_fn,
                           [global_step_t, learning_rate_t, loss_t, y, preds])

    elif mode == tf.estimator.ModeKeys.EVAL:
        alphas_normal = model.arch_parameters()[0]
        alphas_reduce = model.arch_parameters()[1]
        gene_saver = GeneSaver(model, alphas_normal, alphas_reduce)
        eval_hooks = [gene_saver]
        
        (x_train, x_valid) = features
        (y_train, y_valid) = labels
        preds = model(x_valid)
        loss = model._loss(preds, y_valid)
        
        b, w, h, c = y_valid.shape
        y = tf.reshape(tf.cast(y_valid, tf.int64), (b, w, h))
        y = tf.one_hot(y, args.num_classes, on_value=1.0, off_value=0.0)
        
        metric_fn = lambda y, preds: {
            'miou': tf.metrics.mean_iou(
                    labels=y,
                    predictions=tf.round(preds),
                    num_classes=args.num_classes
                ),
        
            'acc': tf.metrics.accuracy(labels=y, 
                                      predictions=tf.round(preds)),
#             'loss': loss
        }
        eval_metric_ops = (metric_fn, [y, preds])
        
    elif mode == tf.estimator.ModeKeys.PREDICT:
        x = features
        y = labels
        preds = model(x)
        prediction_dict = {"predictions": preds}
        export_outputs = {"predict_export_outputs": tf.estimator.export.PredictOutput(outputs = preds)}
    
    # 5. Return EstimatorSpec
    return tf.estimator.tpu.TPUEstimatorSpec(
        mode = mode,
        predictions = prediction_dict,
        loss = loss,
        train_op = train_op,
        eval_metrics = eval_metric_ops,
        export_outputs = export_outputs,
        evaluation_hooks=eval_hooks,
        host_call=host_call
    )

### Estimator

In [16]:
# Create functions to read in respective datasets
def get_train():
    return make_inp_fn2(filename = tf.gfile.Glob('../../datasets/train_search/train-search-*-00008.tfrecords'),
                        mode = tf.estimator.ModeKeys.TRAIN,
                        batch_size = args.train_batch_size)

In [17]:
# Create serving input function
def serving_input_fn():
    feature_placeholders = {
      IMAGE_LOC: tf.placeholder(tf.float32, [None])
    }
    
    feature_placeholders['IMAGES'] = tf.placeholder(tf.float32, [None, args.crop_size[0], args.crop_size[1], args.init_channels])
    
    features = {
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
    }

    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)


In [18]:
tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
      tpu=args.tpu,
      zone=args.zone,
      project=args.project)

In [19]:
config = tf.estimator.tpu.RunConfig(
      cluster=tpu_cluster_resolver,
      model_dir=args.model_dir,
      save_checkpoints_steps=args.save_checkpoints_steps,
      tpu_config=tf.contrib.tpu.TPUConfig(num_shards=8))

In [20]:
estimator = tf.estimator.tpu.TPUEstimator(
    use_tpu=args.use_tpu,
    model_fn=model_fn,
    config=config,
    train_batch_size=args.train_batch_size,
    eval_batch_size=args.eval_batch_size,
    params=vars(args)
)
estimator.evaluate(input_fn = get_train(), steps = 1)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.240.1.18:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2e8d29c5d0>, '_model_dir': 'gs://unet-darts/train-search-ckptss', '_protocol': None, '_save_checkpoints_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tpu_config': TPUConfig(iterations_per_loop=2, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=2, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None, eval_training_input_configuration=2, experimental_host_call_every_n_steps=1), '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_session_creation_time

{'acc': 0.71809894, 'global_step': 3, 'loss': 1.8874016, 'miou': 0.39712805}

In [27]:
input_fn=get_train()

In [ ]:
current_step = estimator.latest_checkpoint()
if(not current_step):
    current_step = 0
else:
    current_step = int(estimator.latest_checkpoint().split('-')[-1])
tf.logging.info('Training for %d steps. Current step %d.' % (args.max_steps, current_step))

start_timestamp = time.time()
while current_step < args.max_steps:
    next_checkpoint = min(current_step + args.steps_per_eval, args.max_steps)
    estimator.train(input_fn=input_fn, max_steps=next_checkpoint)
    current_step = next_checkpoint

    elapsed_time = int(time.time() - start_timestamp)
    tf.logging.info('Finished training up to step %d. Elapsed seconds %d.' % (current_step, elapsed_time))
    tf.logging.info('Starting to evaluate.')

    eval_results = estimator.evaluate(
        input_fn=input_fn,
        steps=args.num_train_examples // args.eval_batch_size
    )

    tf.logging.info('Eval results: %s' % eval_results)

INFO:tensorflow:Training for 10000 steps. Current step 78.
INFO:tensorflow:Querying Tensorflow master (grpc://10.240.1.18:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 11183773590916195805)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 5961164671056845700)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 6457306311337005987)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 4459697946670094731)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 14908122987920093202)
INFO:t

Exception KeyboardInterrupt in <bound method ScopedTFGraph.__del__ of <tensorflow.python.framework.c_api_util.ScopedTFGraph object at 0x7f34e4bb4510>> ignored


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from gs://unet-darts/train-search-ckptss/model.ckpt-80
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 80 into gs://unet-darts/train-search-ckptss/model.ckpt.
INFO:tensorflow:Initialized dataset iterators in 1 seconds
INFO:tensorflow:Installing graceful shutdown hook.
INFO:tensorflow:Creating heartbeat manager for ['/job:worker/replica:0/task:0/device:CPU:0']
INFO:tensorflow:Configuring worker heartbeat: shutdown_mode: WAIT_FOR_COORDINATOR

INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 7 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Enqueue next (2) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (2) batch(es) of 

In [13]:
# Create custom estimator's train and evaluate function
def train_and_evaluate(output_dir, estimator):
    train_spec = tf.estimator.TrainSpec(input_fn = get_train(),
                                    max_steps = 1000)
    exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn = get_train(),
                                  steps = None, throttle_secs=600)
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [14]:
train_and_evaluate(args.model_dir, estimator)

NameError: name 'estimator' is not defined

## Extra Test

In [14]:
criterion = tf.losses.softmax_cross_entropy
model = Network(C=args.init_channels, net_layers=args.num_layers, criterion=criterion, num_classes=args.num_classes)

In [15]:
(x_train, x_valid), (y_train, y_valid) = get_train()(vars(args)).make_one_shot_iterator().get_next()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [16]:
logits = model(x_train)

In [14]:
loss = model._loss(logits, y_train)

(<tf.Tensor 'one_hot:0' shape=(8, 4, 4, 6) dtype=float32>, <tf.Tensor 'network/softmaxConv/softmax/Softmax:0' shape=(8, 4, 4, 6) dtype=float32>)


In [16]:
optimizer = tf.train.GradientDescentOptimizer(0.001)
optimizer.minimize(loss, var_list=model.get_thetas())

<tf.Operation 'GradientDescent' type=NoOp>

In [17]:
y_train

<tf.Tensor 'IteratorGetNext:2' shape=(8, 4, 4, 1) dtype=float32>

In [18]:
logits

<tf.Tensor 'network/softmaxConv/softmax/Softmax:0' shape=(8, 4, 4, 6) dtype=float32>

In [20]:
b, w, h, c = y_train.shape
y = tf.reshape(tf.cast(y_train, tf.int64), (b, w, h))
y = tf.one_hot(y, args.num_classes, on_value=1.0, off_value=0.0)

In [21]:
y

<tf.Tensor 'one_hot:0' shape=(8, 4, 4, 6) dtype=float32>

In [53]:
miou = tf.metrics.mean_iou(
    labels=y,
    predictions=tf.round(logits),
    num_classes=args.num_classes
)

In [50]:
b = tf.metrics.mean_iou(
    labels=tf.constant([[1, 0, 0], [0, 1, 0]]),
    predictions=tf.constant([[1, 0, 0], [0, 0.99999, 0]]),
    num_classes=2
)

In [54]:
local_op = tf.local_variables_initializer()
global_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run([local_op, global_op])
    out = sess.run(miou)